In [1]:
import json
import requests

from googlesearch import search

import praw

from multiprocess import Pool
import time

from g4f.client import Client

In [13]:
query = "best res"

cursor_pos = len(query)
url_friendly_query = query.replace(" ", "%20")

url = f"https://www.google.com/complete/search?q={url_friendly_query}%20reddit&cp={cursor_pos}&client=gws-wiz&xssi=t&hl=en&authuser=0&dpr=2"

res = requests.get(url).text[5:]
parsed = json.loads(res)[0]

In [14]:
suggestions = [element[0].replace("reddit", "").replace(" </b>", "</b>") for element in parsed]

In [15]:
suggestions

['best res<b>ume builder</b>',
 'best res<b>ident evil game</b>',
 'best res<b>ume template</b>',
 'best res<b>istance bands</b>',
 'best res<b>taurants nyc</b>',
 'best res<b>ume writing service</b>',
 'best res<b>taurants chicago</b>',
 'best res<b>taurants montreal</b>',
 'best res<b>p canada</b>',
 'best res<b>taurants in rome</b>']

In [2]:
submission_ids = []
for res in search("best git merge tool reddit", safe=None):
    if "/comments/" in res:
        submission_id = res.split("/")[6]
        submission_ids.append(submission_id)

In [3]:
r = praw.Reddit(
    client_id="2gs3ULbVK-pGh7INla0gWg",
    client_secret="g6S2REUlWw8fBBIgJiaFrqIkZUVICQ",
    user_agent="web:tech.redditrecs:v1.0.0 (by u/matetoes)",
)

In [4]:
submission_ids

['l75rhr',
 '187ph83',
 '158zdir',
 '16xa2q0',
 'mo3pyn',
 'icptpk',
 '6hjm92',
 '18hw0zp',
 '123ha5p',
 'u82rmz',
 'kduahq',
 '15iy7cp']

In [5]:
submissions = []

def jsonify(comment):
    return {
        "body": comment.body,
        "score": comment.score,
        "author": str(comment.author),
        "replies": [jsonify(reply) for reply in comment.replies],
    }

def get(submission_id):
    # let's time how long this takes
    start = time.time()
    comments = []
    submission = r.submission(submission_id)
    submission.comments.replace_more(limit=None)
    # get title and body of submission
    title = submission.title
    selftext = submission.selftext
    for comment in submission.comments:
        comments.append(jsonify(comment))
    result = {
        "title": title,
        "selftext": selftext,
        "comments": comments,
    }
    end = time.time()
    print(f"Submission {submission_id} took {end - start} seconds")
    return result

with Pool(len(submission_ids)) as pool:
    submissions = pool.map(get, submission_ids)

Submission u82rmz took 0.5418558120727539 seconds
Submission kduahq took 0.5784461498260498 seconds
Submission 6hjm92 took 0.6053669452667236 seconds
Submission icptpk took 0.6350729465484619 secondsSubmission l75rhr took 0.6402549743652344 seconds

Submission 123ha5p took 0.6276180744171143 seconds
Submission 187ph83 took 0.7350268363952637 seconds
Submission mo3pyn took 0.7547240257263184 secondsSubmission 158zdir took 0.7592282295227051 seconds

Submission 15iy7cp took 0.7537839412689209 seconds
Submission 16xa2q0 took 0.8493468761444092 seconds
Submission 18hw0zp took 1.7624421119689941 seconds


In [7]:
len(json.dumps(submissions[0]["comments"][0]))

# let's find the max
max_len = 0
max_comment = None
for submission in submissions:
    for comment in submission["comments"]:
        if len(json.dumps(comment)) > max_len:
            max_len = len(json.dumps(comment))
            max_comment = comment

max_len

5713

In [12]:
len(submissions[0]["comments"])

# let's sum all the lengths
print(sum([len(submission["comments"]) for submission in submissions]))

113


In [10]:
def to_input(submission, query):
    return {
        "query": query,
        "title": submission["title"],
        "selftext": submission["selftext"],
        "comment_thread": submission["comments"][0],
    }

In [22]:
example_inputs = []

In [23]:
example_inputs.append(to_input(submissions[0], "best git mergetool"))

In [24]:
print(f"test: {example_inputs[0]}")

test: {'query': 'best git mergetool', 'title': 'Do you use some git mergetool? What should I get to start working seriously with git', 'selftext': "I'm a beginner in git and I want to know if it's good to use a mergetool to use git. I don't find the command line so intuitive to find differences between files. I've heard that a complete GUI for git is not so reliable, is it the same with a mergetool? Is it some tool outside git I need to get or just command line is enough?", 'comments': {'body': 'Git guis are discouraged because they make assumptions about things and hide those complexities from you.\n\nMerge/diff tools have a single purpose - to allow you to see and merge changes in a file.\n\nI use the git command line exclusively, either in bash or PowerShell. But I use a merge tool (BeyondCompare) for diffs and merges.', 'score': 3, 'author': 'Matosawitko', 'replies': [{'body': "> make assumptions about things and hide those complexities from you\n\nFor example? Not arguing just has

In [25]:
example_outputs = []

In [26]:
example_out1 = example_inputs[0].copy()

In [27]:
example_out1

{'query': 'best git mergetool',
 'title': 'Do you use some git mergetool? What should I get to start working seriously with git',
 'selftext': "I'm a beginner in git and I want to know if it's good to use a mergetool to use git. I don't find the command line so intuitive to find differences between files. I've heard that a complete GUI for git is not so reliable, is it the same with a mergetool? Is it some tool outside git I need to get or just command line is enough?",
 'comments': {'body': 'Git guis are discouraged because they make assumptions about things and hide those complexities from you.\n\nMerge/diff tools have a single purpose - to allow you to see and merge changes in a file.\n\nI use the git command line exclusively, either in bash or PowerShell. But I use a merge tool (BeyondCompare) for diffs and merges.',
  'score': 3,
  'author': 'Matosawitko',
  'replies': [{'body': "> make assumptions about things and hide those complexities from you\n\nFor example? Not arguing just 

In [ ]:
system_msg = "You are a helpful assistant."
user_msg = """I will give you a search query (e.g., "best headphones reddit"), a Reddit post that resulted from googling that query, and a comment thread from that post in the form of a JSON object. Reply with a JSON object, mimicking the tree-like structure of the comment thread in the original JSON, such that for each comment in the comment thread, you will list the names of the items that the user recommends (if any) and the names of the items that the user recommends against (if any). Let's go through some examples.

Input: {'query': 'best git mergetool', 'title': 'Do you use some git mergetool? What should I get to start working seriously with git', 'selftext': "I'm a beginner in git and I want to know if it's good to use a mergetool to use git. I don't find the command line so intuitive to find differences between files. I've heard that a complete GUI for git is not so reliable, is it the same with a mergetool? Is it some tool outside git I need to get or just command line is enough?", 'comments': {'body': 'Git guis are discouraged because they make assumptions about things and hide those complexities from you.\n\nMerge/diff tools have a single purpose - to allow you to see and merge changes in a file.\n\nI use the git command line exclusively, either in bash or PowerShell. But I use a merge tool (BeyondCompare) for diffs and merges.', 'score': 3, 'author': 'Matosawitko', 'replies': [{'body': "> make assumptions about things and hide those complexities from you\n\nFor example? Not arguing just hasn't been my experience", 'score': 2, 'author': 'olets', 'replies': [{'body': "Which ones are you familiar with? I've used git gui/gitk, Sourcetree, Github Desktop, and Tortoise. They all vary quite a bit in terms of features, etc, but in general you have to understand the underlying git commands to use them well. At which point you might as well just use the command line.", 'score': 2, 'author': 'Matosawitko', 'replies': [{'body': 'I\'m a Mac user so not Tortoise but the rest of those and others - Git UIs are something I geek out about, so I try to play with all that I can. (Though as I said in another comment GitHub Desktop is so minimally featured and idiomatic I don\'t even really consider it a "Git UI"! More an app for doing a few interactions with Git repos.)\n\nRe needing to know underlying Git commands: this probably comes down to how different people\'s minds work. When I was starting out I found Sourcetree hugely helpful for making sense of basic concepts like staged files, commits, branches. (Later I returned to it to help make sense other things, particularly around history rewriting where the visual confirmation and being able to see everything regardless of whether I\'d know to look for it is helpful for me.) I\'ve seen UIs help other beginners make sense of things too, so I always encourage at least trying it out.', 'score': 2, 'author': 'olets', 'replies': []}]}, {'body': "One example that comes to mind is Visual Studio. They originally used their own language and have been slowly moving to use gits official language. Recently they even pulled all of the git tools from the team panel into their own panel.\n\nThe old GitHub tool also had many assumptions but I haven't used that tool on a while so I don't know what it's current state is.", 'score': 0, 'author': 'am385', 'replies': [{'body': "Gotcha. Agreed on both counts — wasn't thinking of VSCode's Git support as a Git UI, but absolutely GitHub Desktop is (was) extreme on both assumptions and limiting what you can do (so extreme that I too haven't touched it in a long time)", 'score': 1, 'author': 'olets', 'replies': []}]}]}, {'body': "BeyondCompare isn't specifically for Git, right?", 'score': 1, 'author': 'Rafaelchavez', 'replies': [{'body': "Right. It's a general difference tool that is configurable to use with git.", 'score': 1, 'author': 'Matosawitko', 'replies': []}]}, {'body': "I'm a developer on gitkraken, so perhaps I'm biased, but I disagree. Some guis hide complexity sure, but they can also improve visibility to the internal state of the repo, which makes learning git way easier. This is where they really shine in my opinion. Github desktop is a poor example, look instead to Gitkraken, Fork, or Sourcetree. All of them provide feedback on how your actions affect your commits and the state of your repo that's hard to get on the CLI alone.\n\nTake the question of rebase vs merge for example. It's a question beginners ask a lot, even though they're pretty different in terms of what they do. It's hard to give a precise beginner-friendly verbal explanation of the difference, in part because any verbal explanation of a git thing is bound to be extremely jargon-heavy, but it's extremely obvious how they're different by just trying both and watching what they do to the graph.", 'score': 1, 'author': 'axosoft-chuckd', 'replies': []}]}}


"""

client = Client()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": system_msg},
              {"role": "user", "content": user_msg}],
)
print(response.choices[0].message.content)